# Treinamento de modelos LDA para análise de tópicos em comentários do subreddit r/mturk
Este notebook treina uma série de modelos de tópicos a serem usados numa análise exploratória corpus de comentários do subreddit /r/mturk coletado para a dissertação através do `/tracer`.
Faremos isso com a ajuda do LDA (Latent Dirichlet Allocation, treinando diferentes versões com uma variede de parâmetros.
Os modelos treinados serão usados como ferramenta de análise exploratória. No notebook `vis.ipynb`, faremos a visualização dos tópicos encontrados, e análise dos tópicos será feita na dissertação.

## Carregando os dados

In [1]:
import pandas as pd

df = pd.read_csv('../data/corpus.csv')
df = df[['created_utc', 'body']]
df = df.dropna(subset=['body'])
df.head()

/var/folders/29/2dgn6km56qb307n7j1k2_jyr0000gn/T/ipykernel_98971/815672776.py:3: DtypeWarning: Columns (7,11,23,25,26,29,32,34,37,38,40,46,49,50,54,56,57,59,60,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/corpus.csv')


,created_utc,body
0,1470773823,[This](https://greasyfork.org/en/scripts/10615...
1,1470773732,Can I ask where are you from? I'm a worker fro...
2,1470773686,2.4.9
3,1470773635,Which version are you using? I have settings ...
4,1470773208,[deleted]


## Treinamento com LDA
Usamos as bibliotecas `gensim` e `nltk` para fazer o treinamento dos modelos LDA.

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from gensim import models

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vitorgomes/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vitorgomes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Pré-processamento de texto: remoção de stopwords e pontuação, lematização)

Neste passo, pré-processamos os dados de texto removendo stopwords e pontuação, e lematizando as palavras.
Também removemos informações sensíveis como nomes de usuários e URLs, minimizando o risco de re-identificação e violações de privacidade.

In [3]:
import re

stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):

    # Passo 1: Remove URLs due to privacy concerns
    doc = re.sub(r"http[s]?://\S+|www\.\S+", "", doc)
    # Passo 2: Remove Reddit-style usernames (e.g., u/username) (privacy concerns)
    doc = re.sub(r"\bu/\w+\b", "", doc)
    # Passo 3: Remove email addresses (privacy concerns)

    # Limpeza final: Remove stopwords/pontuação e lematiza palavras
    doc = re.sub(r"\b\w+@\w+\.\w+\b", "", doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())

    return normalized

doc_clean = [clean(doc).split() for doc in df['body']]

### Gerando o dicionário e o corpus

In [4]:
from gensim.corpora import Dictionary

# Cria uma representação de dicionário dos documentos
dictionary = Dictionary(doc_clean)

# Filtra o dicionário para remover termos que aparecem em menos de 2 documentos e em mais de 95% dos documentos
dictionary.filter_extremes(no_below=2, no_above=0.95)

# Representa os documentos na forma bag-of-words
corpus = [dictionary.doc2bow(doc) for doc in doc_clean]

### Ajustando o modelo

#### Modelo LDA com 10 tópicos

In [5]:
lda_10t = models.LdaModel(corpus=corpus, num_topics=10, id2word=dictionary, passes=10)

#### Modelo LDA com 100 tópicos

In [6]:
lda_100t = models.LdaModel(corpus=corpus, num_topics=100, id2word=dictionary, passes=10)

### Salvando os modelos para o disco

In [13]:
import os
from gensim.corpora import MmCorpus

os.chdir("../eda")

dictionary.save('./models/dictionary.gensim')
MmCorpus.serialize('./models/corpus.mm', corpus)

In [15]:

lda_10t.save('models/lda_10t/lda_10t.gensim')

In [16]:

lda_100t.save('models/lda_100t/lda_100t.gensim')